In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install beautifulsoup4

print('Libraries imported.')

<h1> Scrape table to result var </h1>

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

r = requests.get(url)
html = r.text

soup = BeautifulSoup(html)
table = soup.find('table', {"class": "wikitable sortable"})
rows = table.find_all('tr')
data = []
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

result = pd.DataFrame(data, columns=['Postal Code', 'Borough', 'Neighbourhoods'])

<h1> Clean results and copy to new DF </h1>

In [ ]:
clean_res = result[result['Borough'] != 'Not assigned']

<h1> Force neighbourhood name in case it is Not Assigned </h1>

In [ ]:
mask = (clean_res['Neighbourhoods'] == 'Not Applicable')
clean_res.loc[mask,'Neighbourhoods'] = clean_res.loc[mask,'Borough']

In [ ]:
clean_res.shape

<h1> Import geo locations for 2nd assignment </h1>

In [ ]:
geo_loc = pd.read_csv('https://cocl.us/Geospatial_data')
geo_loc.head()

<h1> Left Join first df with the new geo locations </h1>

In [ ]:
clean_res_full = pd.merge(clean_res,  
                      geo_loc,  
                      on ='Postal Code',  
                      how ='left') 
clean_res_full 

In [ ]:
clean_res_full = clean_res_full.rename(columns={'Neighbourhoods': "Neighborhood"})

<h1> Foursqaure Credentials </h1>

In [ ]:
CLIENT_ID = 'C0S23INNSHUDFD4ZYYBSZARVQPA5UM1RHYQJ1LWFYLGOONNM' # your Foursquare ID
CLIENT_SECRET = 'U5PVAZ0IBYJV3XBBFRA4GPW3WLYNE4PBTFKW4FDJHU5MUGHM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
RADIUS = 500

<h1> Function for venues data </h1>

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h1> Loop to get all venues data for all Neighbourhoods </h1>

In [ ]:
canada_venues = getNearbyVenues(names=clean_res_full['Neighborhood'],
                                   latitudes=clean_res_full['Latitude'],
                                   longitudes=clean_res_full['Longitude']
                                  )


<h1> Pivot data by Venue Category </h1>

In [ ]:
t_venues = pd.crosstab(all_venues['Neighborhood'], all_venues['Venue Category'])

<h1> Get most common venues to new dataframe </h1>

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = t_venues['Neighborhood']

for ind in np.arange(t_venues.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t_venues.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<h1> Clustering </h1>

In [ ]:
# set number of clusters
kclusters = 5

t_venues_cluster = t_venues.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_venues_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop(columns='Neighborhood')
neighborhoods_venues_sorted['Neighborhood'] = neighborhoods_venues_sorted.index

In [ ]:
neighborhoods_venues_sorted.dtypes

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

<h1> Merge Data in order to get lat and lang </h1>

In [ ]:
canada_merged = clean_res_full

# merge neighborhoods_venues_sorted with canada_merged to add latitude/longitude for each neighborhood
canada_full = canada_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#Clean Nan
canada_full = canada_full[canada_full['Cluster Labels'].notna()]

canada_full.head() # check the last columns!

<h1> Initialize geo location and show map </h1>

In [ ]:
address = 'Canada, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(canada_full['Latitude'], canada_full['Longitude'], canada_full['Neighborhood'], canada_full['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters